In [ ]:
from countminsketch import CountMinSketch
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import roc_auc_score
from operator import add

np.random.seed(10)
p= 10000
w = 4

waveform_data = pd.read_csv("sanitized_waveform.data",header = None)
waveform_data_exc = waveform_data.drop(21, 1)

seed_np = np.random.seed(1)
s = 1000.0                                                      #sample size
sqrt_s = np.sqrt(s)

ensemble_scores = [0] * waveform_data_exc.shape[0]

m = 300
z = m
while(z>0):
    scores = []
    f = np.random.uniform(1.0/sqrt_s,1.0 - (1.0/sqrt_s))            #locality parameter
    
    cm_sketch = CountMinSketch(p,w)
    d = waveform_data.shape[1] #dimension of the dataset
    random_vector = np.random.uniform(0,f,d-1)
    #print(random_vector)
    
    #np.random.seed(1)
    base =  np.maximum(2, 1/f)
    r = int(np.random.uniform(1.0+(0.5*math.log(s,base)),math.log(s,base)))
    
    #print waveform_data_exc.shape[1]
    waveform_randomly = waveform_data_exc.sample(n=r,axis = 1)
    #print(waveform_randomly.head())
    #print waveform_randomly.shape
    
    sample_data = waveform_data_exc.sample(n = int(s))
    
    indexes_sample = sample_data.index
    
    included_cols = waveform_randomly.columns

    col_max_whole = np.max(sample_data,axis = 0)
    col_min_whole = np.min(sample_data,axis = 0)

    Y_cap = (sample_data - col_min_whole)/(col_max_whole - col_min_whole)

    #print included_cols
    for col in Y_cap:
        if col not in included_cols:
            Y_cap[col] = -1
        else:
            Y_cap[col] = np.floor((Y_cap[col] + random_vector[col] )/ f)
    #print(Y_cap.head())
    #print Y_cap.shape
    
    data_tuples = [tuple(x) for x in Y_cap.to_records(index=False)]
    #print(data_tuples[0])
    for i in data_tuples:
        cm_sketch.add(i)
        
    Y_whole = waveform_data_exc
    norm_whole_data = (Y_whole - col_min_whole) / (col_max_whole - col_min_whole)
    
    for col in Y_whole:
        if col not in included_cols:
            norm_whole_data[col] = -1
        else:
            norm_whole_data[col] = np.floor(norm_whole_data[col] + random_vector[col] / f)
        
    whole_data_tuples = [tuple(x) for x in norm_whole_data.to_records(index=False)]
    #print len(whole_data_tuples[0])
    #print len(whole_data_tuples)
    for i in whole_data_tuples:
        cm_sketch.add(i)
        
    sample_data_tuples = [tuple(x) for x in Y_cap.to_records(index=False)]
    #print (sample_data_tuples[0])
    #print (whole_data_tuples[0])
    #print sample_data_tuples
    #(-1, -1, -1, -1, -1, -1, -1, 0.0, 1.0, 1.0, 1.0, -1, -1, 1.0, -1, -1, -1, -1, -1, -1, 2.0)
    #print whole_data_tuples[3288]
    for i in whole_data_tuples:
        if(i in data_tuples):
            scores.append(np.math.log(cm_sketch.query(i),2)+1)
        else:
            scores.append(np.math.log(cm_sketch.query(i),2))
    
    ensemble_scores = map(add, ensemble_scores, scores)
    z = z - 1
    print (z)
    
ensemble_scores = [x / m for x in ensemble_scores]

y_true = waveform_data[21]
y_true = y_true.values.tolist()
for i in range(0, len(y_true)):
    if y_true[i] == 2:
        y_true[i] = 1
roc = roc_auc_score(y_true, ensemble_scores)
print "Area under TPR curve: ", roc